In [1]:
import time, re, csv, requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
def cut(x):
    item = x.get_text().replace('\n', '').replace('\u200b', '')
    if len(item):
        return True
    return False


def select_title(soup, selector):
    title = soup.select_one(selector)
    if title is None:
        return None
    return title.get_text().strip()

In [ ]:
with open('search_page.html', 'r', encoding='utf-8') as f:
    d = f.read()
soup = BeautifulSoup(d, "lxml")
a_tags = soup.find_all("a", attrs={"class": "api_txt_lines"})
blog_links = [a_tag.attrs.get('href') for a_tag in a_tags]

In [3]:
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
options = Options()
options.add_argument(f"--user-agent={ua}")
service=Service(ChromeDriverManager().install())

In [5]:
browser = webdriver.Chrome(service=service, options=options)

filename = 'blog_crawl_origin.csv'
with open(filename, 'w', encoding='utf-8-sig', newline="") as f:
    writer = csv.writer(f)
    # 컬럼 명
    columns = "line, url, site, title, author, created_date, text".split(', ')
    writer.writerow(columns)
    line = 0
    for i, blog_link in enumerate(blog_links):
        if not blog_link.startswith('https://blog.naver.com'): continue
        browser.get(blog_link)
        browser.switch_to.frame('mainFrame')
        soup = BeautifulSoup(browser.page_source, "lxml")
        
        first_title = select_title(soup, 'div.se-module.se-module-text.se-title-text')
        second_title = select_title(soup, 'div.se_editView.se_title')
        
        if first_title: title = first_title
        elif second_title: title = second_title
        else: continue
        
        author = soup.select_one('span.nick').get_text()
        created_date = soup.select_one('span.se_publishDate').get_text()
        texts = [item.get_text().replace('\n', '').replace('\u200b', '') for item in soup.select('div.se-component.se-text') if cut(item)]
        text = '\n'.join(texts)
        
        if text == '': continue
        
        record = [line, blog_link, 'naver blog', title, author, created_date, text]
        writer.writerow(record)
        
        line += 1
        
browser.close()

In [57]:
filename = 'blog_crawl_origin.csv'
with open(filename, 'r', encoding='utf-8-sig') as f:
    lines = csv.reader(f)
    with open('blog_crawl.csv', 'w', encoding='utf-8-sig', newline='') as fw:
        writer = csv.writer(fw)
        writer.writerow(['index', 'created_date', 'text'])
        for i, line in enumerate(list(lines)[1:]):
            index = line[0]
            created_date = line[5]
            txt = line[3] + '\n' + line[6]
            writer.writerow([index, created_date, txt])